# Reading, manipulating, and writing data with Astropy

TODO:

unified i/o: `astropy.io`

if tabular data, returns an astropy table class

let's learn about the class in general, then do some specific examples with ascii and FITS tables


TODO: link to relevant sections of documentation
For more information about the features presented below, you can read the
[astropy.table](http://docs.astropy.org/en/stable/table/index.html) documentation.

In [1]:
# Imports we'll need throughout the tutorial:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

from astropy.table import Table
from astropy.io import ascii, fits

# Astropy Tables

The astropy [Table](http://docs.astropy.org/en/stable/table/index.html) class provides an extension of NumPy structured arrays for storing and manipulating heterogeneous tables of data. [Table](http://docs.astropy.org/en/stable/table/index.html) supports many convenient features that may be familiar to users of other packages that support table data (e.g., `Pandas`). To highlight a few:

- `Table` objects can be modified in place by adding or removing columns, changing column names, or adding new rows of data.
- `Table`s support dealing with missing values.
- `Table`s support database operations like joins, concatenation, and grouping.
- `Table`s support Astropy [Units and Quantities](http://astropy.readthedocs.org/en/stable/units/index.html), especially with the `QTable` class.

## Creating an astropy `Table` from scratch


Table objects can be constructed in a variety of ways:

- By reading an existing table from a file or web URL
- By adding columns of data one by one
- By adding rows of data one by one
- From an existing data structure in memory, i.e. from a:

  - List of data columns
  - Dict of data columns
  - List of row dicts
  - Numpy homgeneous array or structured array
  - List of row records
  
See the documentation section on [Constructing a table](http://astropy.readthedocs.org/en/stable/table/construct_table.html) for the gory details and plenty of examples.

Let's create our first table object by creating an empty table and start by adding a single column:

In [2]:
tbl = Table()
tbl['name'] = ['Graham Chapman', 'John Cleese', 'Terry Gilliam', 
               'Eric Idle', 'Terry Jones', 'Michael Palin']

In this case, the column name is given as a string index to the table object, and the data type and number of rows are inferred directly from the data. Once the table has a column, however, any new columns we add must have the same number of rows:

In [3]:
tbl['age'] = [np.nan, 78, 77, 74, 75, 74]

# whereas, this would fail because it isn't the same length as the table:
# tbl['age'] = [77, 77]

#### Displaying or viewing tables

An instantiated table object can be viewed in a number of ways. In a Jupyter notebook, showing the table will produce a nice HTML representation of the table:

In [4]:
tbl

name,age
str14,float64
Graham Chapman,nan
John Cleese,78.0
Terry Gilliam,77.0
Eric Idle,74.0
Terry Jones,75.0
Michael Palin,74.0


If you did the same in a terminal session you get a different view that isn't as pretty but does give a bit more information about the table. In a terminal session, it would look like this:

In [5]:
print(tbl)

     name      age 
-------------- ----
Graham Chapman  nan
   John Cleese 78.0
 Terry Gilliam 77.0
     Eric Idle 74.0
   Terry Jones 75.0
 Michael Palin 74.0


We can access the table column names and data types using the `colnames` and `dtype` attributes:

In [6]:
tbl.colnames

['name', 'age']

In [7]:
tbl.dtype

dtype([('name', '<U14'), ('age', '<f8')])

For long tables, there is also a handy ``show_in_notebook()`` method that allows more interactive exploration like searching, and paged views of the data:

In [8]:
tbl.show_in_notebook()

idx,name,age
0,Graham Chapman,nan
1,John Cleese,78.0
2,Terry Gilliam,77.0
3,Eric Idle,74.0
4,Terry Jones,75.0
5,Michael Palin,74.0


#### Accessing sections of the data

We can access the columns and rows the same as we would with numpy structured arrays: string indices access column names, and numeric indices access rows. In the examples below, notice that the returned objects are `Column`, `Row` or `Table` objects depending on the selection:

In [9]:
tbl['name']  # access entire column

Graham Chapman
John Cleese
Terry Gilliam
Eric Idle
Terry Jones
Michael Palin


In [10]:
tbl['name'][1] # row index 1 of column

'John Cleese'

In [11]:
tbl[1]  # row object for row index 1

name,age
str14,float64
John Cleese,78.0


In [12]:
tbl[1]['name']  # name column of row index 1

'John Cleese'

In [13]:
tbl[1:3]  # select a range of rows and return a new table

name,age
str14,float64
John Cleese,78.0
Terry Gilliam,77.0


In [14]:
tbl[[1, 3]] # select specific row indices and return a new table

name,age
str14,float64
John Cleese,78.0
Eric Idle,74.0


#### Masking rows from a table

Like Numpy arrays, we can also use boolean arrays or masks to filter tables. For example, to get the subset of rows where `'age'` is < 75 and finite:

In [15]:
mask = (tbl['age'] < 75) & np.isfinite(tbl['age'])
tbl[mask]  # Create a new table with just the rows where the mask is True

/Users/adrian/anaconda/lib/python3.5/site-packages/astropy/table/column.py:929: RuntimeWarning: invalid value encountered in less
  return getattr(self.data, oper)(other)


name,age
str14,float64
Eric Idle,74.0
Michael Palin,74.0


#### Modifying a table

Once the table exists with defined columns there are a number of ways to modify the table in place.  These are fully documented in the section [Modifying a Table](http://astropy.readthedocs.org/en/stable/table/modify_table.html#modifying-a-table).

We already saw an example above of how to add new columns, but let's add another column. We can also add new rows using the [add_row()](http://astropy.readthedocs.org/en/stable/api/astropy.table.Table.html#astropy.table.Table.add_row) method:

In [16]:
tbl['british'] = [True, True, False, True, True, True]

In [17]:
tbl.add_row(('Eddie Izzard', 55, True))  # add a new row to the end
tbl

name,age,british
str14,float64,bool
Graham Chapman,nan,True
John Cleese,78.0,True
Terry Gilliam,77.0,False
Eric Idle,74.0,True
Terry Jones,75.0,True
Michael Palin,74.0,True
Eddie Izzard,55.0,True


Notice that the `age` column really has too many output digits given that the age should be an integer.  We can change the display behavior by setting the format using a Python format string:

In [18]:
tbl['age'].format = '%.0f'
tbl

name,age,british
str14,float64,bool
Graham Chapman,nan,True
John Cleese,78,True
Terry Gilliam,77,False
Eric Idle,74,True
Terry Jones,75,True
Michael Palin,74,True
Eddie Izzard,55,True


#### Converting a table to a numpy structured array

Sometimes you may not want or be able to use a `Table` object and prefer to work with a plain numpy array.  This is easily done by passing the table to the `np.array()` constructor.  

Note that this makes a copy of the data.  If you have a huge table and don't want to waste memory, supply `copy=False` to the constructor, but be warned that changing the output numpy array will change the original table.

In [19]:
np.array(tbl)

array([('Graham Chapman',  nan,  True), ('John Cleese',  78.,  True),
       ('Terry Gilliam',  77., False), ('Eric Idle',  74.,  True),
       ('Terry Jones',  75.,  True), ('Michael Palin',  74.,  True),
       ('Eddie Izzard',  55.,  True)], 
      dtype=[('name', '<U14'), ('age', '<f8'), ('british', '?')])

---

# Reading tabular data into `Table` objects

Tabular data in any format supported by the [unified input/ouput interface](http://docs.astropy.org/en/latest/io/unified.html) can be read in as a `Table` object using the [Table.read()](http://astropy.readthedocs.org/en/stable/api/astropy.table.Table.html#astropy.table.Table.read) method. Some notable examples are FITS files with a table, CSV or other delimited ASCII table, or Latex tables. Let's see some examples using files included with this repository:

In [20]:
!ls -l example_table_data*

-rw-r--r--  1 adrian  staff   2906 Nov 23 12:58 example_table_data.csv
-rw-r--r--@ 1 adrian  staff  17280 Nov 23 12:57 example_table_data.fits
-rw-r--r--  1 adrian  staff   2599 Nov 23 13:03 example_table_data.tex
-rw-r--r--  1 adrian  staff   2906 Nov 23 13:03 example_table_data.txt


In [21]:
tbl1 = Table.read('example_table_data.fits')
tbl1

RAJ2000,DEJ2000,_2MASS,Jmag,e_Jmag,Hmag,e_Hmag,Kmag,e_Kmag,Qflg,Rflg,Bflg,Cflg,Xflg,Aflg
deg,deg,,mag,mag,mag,mag,mag,mag,,,,,,
float64,float64,str17,float32,float32,float32,float32,float32,float32,str3,str3,str3,str3,uint8,uint8
10.439434,89.977051,00414546+8958373,15.36,0.068,14.871,0.101,14.606,0.114,AAB,222,111,000,0,0
23.510793,89.977364,01340259+8958385,16.475,0.179,15.874,nan,16.14,nan,CUU,200,100,000,0,0
74.304832,89.979912,04571315+8958476,16.703,0.138,17.575,nan,16.318,nan,BUU,200,100,000,0,0
78.158658,89.989044,05123807+8959205,15.715,0.077,15.013,0.099,14.672,0.094,AAA,222,111,000,0,0
78.403255,89.985268,05133678+8959069,18.6,nan,16.479,nan,15.078,0.125,UUB,002,001,000,0,0
79.221887,89.972488,05165325+8958209,16.056,0.127,15.654,0.216,15.353,0.216,BCC,222,111,000,0,0
86.793746,89.969536,05471049+8958103,16.78,0.149,17.081,nan,15.471,nan,BUU,200,100,000,0,0
110.92212,89.983841,07234130+8959018,18.471,nan,15.569,0.144,17.006,nan,UBU,020,010,000,0,0


The `Table.read()` method will try to figure out what type of file and data structure the input file contains:

In [22]:
tbl2 = Table.read('example_table_data.csv')

but sometimes you have to specify other options to help it out:

In [23]:
tbl3 = Table.read('example_table_data.tex', format='ascii.aastex')

In [24]:
tbl4 = Table.read('example_table_data.txt', format='ascii', delimiter='|')

A number of common astronomical ascii formats are supported such as IPAC, sextractor, daophot, and CDS - see [the documentation](http://docs.astropy.org/en/latest/io/unified.html) for a full list, or the docstring:

In [25]:
Table.read?

--- 

# Writing `Table` objects

To write a table object to disk as a binary or ascii file, use the [Table.write()](http://astropy.readthedocs.org/en/stable/api/astropy.table.Table.html#astropy.table.Table.write) method. This acts just like the `.read()` method, in that in some cases it can automatically guess the format from the filename, but may require specifying the format or other parameters (e.g., delimiter).

An example writing a FITS file:

In [33]:
tbl1.write('test.fits', overwrite=True)

And with ASCII files of various formats:

In [35]:
# VO table
tbl1.write('test.vot', format='votable', overwrite=True)
!head -n 30 test.vot

<?xml version="1.0" encoding="utf-8"?>
<!-- Produced with astropy.io.votable version 2.0.2
     http://www.astropy.org/ -->
<VOTABLE version="1.3" xmlns="http://www.ivoa.net/xml/VOTable/v1.3" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://www.ivoa.net/xml/VOTable/v1.3">
 <RESOURCE type="results">
  <TABLE>
   <FIELD ID="RAJ2000" datatype="double" name="RAJ2000" unit="deg"/>
   <FIELD ID="DEJ2000" datatype="double" name="DEJ2000" unit="deg"/>
   <FIELD ID="_2MASS" arraysize="17" datatype="unicodeChar" name="_2MASS"/>
   <FIELD ID="Jmag" datatype="float" name="Jmag" unit="mag"/>
   <FIELD ID="e_Jmag" datatype="float" name="e_Jmag" unit="mag"/>
   <FIELD ID="Hmag" datatype="float" name="Hmag" unit="mag"/>
   <FIELD ID="e_Hmag" datatype="float" name="e_Hmag" unit="mag"/>
   <FIELD ID="Kmag" datatype="float" name="Kmag" unit="mag"/>
   <FIELD ID="e_Kmag" datatype="float" name="e_Kmag" unit="mag"/>
   <FIELD ID="Qflg" arraysize="3" datatype="unico

In [36]:
tbl1.write('test.dat', format='ascii')

In [37]:
tbl1.write('test.csv', format='ascii.csv')

In [39]:
tbl1.write('test.ipac', format='ascii')

While the above is often sufficient, it's important to know that the base-line format does *not* support quite a few of the "advanced" features, like inclusion of units.  To do that you'll likely want to use the astropy table "enhanced" CSV:

In [40]:
tbl1.write('test.ecsv', format='ascii.ecsv')

---

# Table columns with units

TODO:


---

TODO: check on below

# Exercises


### Read the data

To start with, read in the two data files representing the master source list and observations source list.  The fields for the two tables are respectively documented in:

- [master_sources](http://cxc.harvard.edu/csc/columns/master.html)
- [obs_sources](http://cxc.harvard.edu/csc/columns/persrc.html)

In [ ]:
master_sources = Table.read('cdfs_master_sources.fits')
obs_sources = Table.read('cdfs_obs_sources.fits')

**`master_sources`**

Each distinct X-ray source identified on the sky is represented in the catalog by a single "master source" entry and one or more "source observation" entries, one for each observation in which the source has been detected. The master source entry records the best estimates of the properties of a source, based on the data extracted from the set of observations in which the source has been detected.  The subset of fields in our exercise table file are:

Name | Description
------ | ------------
msid  | Master source ID
name  | Source name in the Chandra catalog
ra  | Source RA (deg)
dec | Source Dec (deg)

**`obs_sources`**

The individual source entries record all of the properties about a detection extracted from a single observation, as well as associated file-based data products, which are observation-specific.  The subset of fields in our exercise table file are:

Name | Description
------ | ------------
obsid | Observation ID
obi | Observation interval
targname | Target name
gti_obs | Observation date
flux_aper_b | Broad band (0.5 - 7 keV) flux (erg/cm2/sec)
src_cnts_aper_b | Broad band source counts
ra_b | Source RA (deg)
dec_b | Source Dec (deg)
livetime | Observation duration (sec)
posid | Position ID
theta | Off-axis angle (arcmin)
msid | Master source ID

### Exploring the data
Do the following to explore the two tables:

- Display the data for each table in IPython notebook using the normal way of showing the value of a variable.
- Get a list of the column names for each table.  *Hint*: use `<TAB>` completion to easily discover all the attributes and methods, e.g. type `master_sources.` and then hit the `<TAB>` key.
- Find the length of each table.
- Find the column datatypes for each table.

Normally one displays a table in IPython notebook by entering the variable name in a cell and pressing `shift-Enter`.  In a terminal session the default method is using something like `print(my_table)`.  In both cases the `Table` object prefers to display only a screenful of data to prevent having a zillion lines of output if the table is huge.  If you really want to see all the data you can use the [Table.pprint](http://astropy.readthedocs.org/en/stable/api/astropy.table.Table.html#astropy.table.Table.pprint) method. If you are using a Jupyter notebook interface, try the `show_in_notebook()` method.

- Display all the rows of the `master_sources` table using its `pprint()` method.
- If you are working in a regular terminal window (not IPython notebook), try the `more()` method as well.

### Modifying tables
For our analysis we don't actually need the `obi` (observation interval) column in the `obs_sources` table.

- Remove the `obi` column from the `obs_sources` table.

The `gti_obs` column name is a bit obscure (GTI is a good time interval, FWIW).

- Rename the `gti_obs` column to `obs_date`.

It would be nice to have a count rate in addition to the source counts.

- Add a new column `src_rate_aper_b` which is the source counts divided by observation duration in sec.

Some of the sources have a negative net flux in the broad band

### Looking at the observation source data
For each source detected in an individual observation (in the `obs_sources` table), let's look at the source flux values.

- Use the matplotlib [`hist()`]( http://matplotlib.org/api/pyplot_api.html?highlight=pyplot.hist#matplotlib.pyplot.hist) function to make a histogram of the source fluxes.  Since the fluxes vary by orders of magnitude,
  use the `numpy.log10` to put the fluxes in log space.

- Also make the same plot but using only sources within 4 arcmin of the center.  *HINT*: use a boolean mask to select values of `theta` that are less than 4.0.